In [1]:
# Data preparation
import pandas as pd
import numpy as np
import os
import zipfile
from pathlib import Path
import cv2

# Paths
zip_file_path = r"C:\Users\shree\Downloads\archive (13).zip"
extracted_path = r"/content/braille_dataset"

# Unzip the dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

In [2]:
# Load images
image_dir = Path(r'/content/braille_dataset/Braille Dataset/Braille Dataset')
dir_list = list(image_dir.glob('*.jpg'))
image_count = len(dir_list)
print(f"Total images: {image_count}")

# Read images
images = []
name_list = []
for file_path in dir_list:
    I = cv2.imread(str(file_path))
    images.append(I)
    name_list.append(os.path.basename(file_path)[0])

# Convert to numpy arrays
images_list = np.array(images)
name_list = np.array(name_list).T

# Normalize images
images_list = images_list / 255.0
print(images_list[0])


Total images: 1560
[[[0.76078431 0.76078431 0.76078431]
  [0.8        0.8        0.8       ]
  [0.80784314 0.80784314 0.80784314]
  ...
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]]

 [[0.75686275 0.75686275 0.75686275]
  [0.69411765 0.69411765 0.69411765]
  [0.7372549  0.7372549  0.7372549 ]
  ...
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]]

 [[0.78039216 0.78039216 0.78039216]
  [0.79607843 0.79607843 0.79607843]
  [0.76470588 0.76470588 0.76470588]
  ...
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]]

 ...

 [[0.76862745 0.76862745 0.76862745]
  [0.77254902 0.77254902 0.77254902]
  [0.76470588 0.76470588 0.76470588]
  ...
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]]

 [[0.77254902 0.77254902 0.77254902]
  [0.77647059 0.77

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode labels
le = LabelEncoder()
name_list = le.fit_transform(name_list)

# Split data
X_train, X_test, y_train, y_test = train_test_split(images_list, name_list, test_size=0.2, random_state=42)
class_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [5]:
import keras

# Define the model basic CNN
model = keras.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(28,28,3)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=512, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=26, activation="softmax")
])

model.compile(optimizer="Adam", loss="SparseCategoricalCrossentropy", metrics=["sparse_categorical_accuracy"])

C:\Users\shree\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train the model
history = model.fit(x=X_train, y=y_train, epochs=40, validation_split=0.3)

# Model summary
model.summary()

Epoch 1/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 161ms/step - loss: 3.8642 - sparse_categorical_accuracy: 0.1871 - val_loss: 3.3768 - val_sparse_categorical_accuracy: 0.0427
Epoch 2/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - loss: 1.5063 - sparse_categorical_accuracy: 0.5676 - val_loss: 4.7811 - val_sparse_categorical_accuracy: 0.0427
Epoch 3/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - loss: 0.8411 - sparse_categorical_accuracy: 0.7285 - val_loss: 5.4432 - val_sparse_categorical_accuracy: 0.0427
Epoch 4/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 130ms/step - loss: 0.5529 - sparse_categorical_accuracy: 0.8213 - val_loss: 6.7621 - val_sparse_categorical_accuracy: 0.0427
Epoch 5/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step - loss: 0.4674 - sparse_categorical_accuracy: 0.8525 - val_loss: 8.3356 - val_sparse_categorical_accuracy: 0.0427
Epoch 6/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 131ms/step - loss: 0.2818 - sparse_categorical_accuracy: 0.9135 - val_loss: 11.8259 - val_sparse_categorical_accuracy: 0.0427
Ep

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 7, 7, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 3, 3, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 3, 3, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 26)                  │          13,338 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,696,528 (17.92 MB)

 Trainable params: 1,565,210 (5.97 MB)

 Non-trainable params: 896 (3.50 KB)

 Optimizer params: 3,130,422 (11.94 MB)

In [8]:
# Model with Transpose convolution
model_transpose = keras.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(28,28,3)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
    keras.layers.Conv2DTranspose(filters=64, kernel_size=(3,3), padding='same', activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(units=512, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=26, activation="softmax")
])

model_transpose.compile(optimizer="Adam", loss="SparseCategoricalCrossentropy", metrics=["sparse_categorical_accuracy"])

In [9]:
# Train the model with transpose convolution
history_transpose = model_transpose.fit(x=X_train, y=y_train, epochs=40, validation_split=0.3)

Epoch 1/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 185ms/step - loss: 3.2132 - sparse_categorical_accuracy: 0.0936 - val_loss: 3.2474 - val_sparse_categorical_accuracy: 0.0480
Epoch 2/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 132ms/step - loss: 1.7699 - sparse_categorical_accuracy: 0.4731 - val_loss: 3.6759 - val_sparse_categorical_accuracy: 0.0427
Epoch 3/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 120ms/step - loss: 1.0467 - sparse_categorical_accuracy: 0.7019 - val_loss: 4.4616 - val_sparse_categorical_accuracy: 0.0427
Epoch 4/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - loss: 0.6007 - sparse_categorical_accuracy: 0.8345 - val_loss: 5.1935 - val_sparse_categorical_accuracy: 0.0427
Epoch 5/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - loss: 0.3091 - sparse_categorical_accuracy: 0.9059 - val_loss: 6.5129 - val_sparse_categorical_accuracy: 0.0427
Epoch 6/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - loss: 0.2380 - sparse_categorical_accuracy: 0.9341 - val_loss: 8.6809 - val_sparse_categorical_accuracy: 0.0427
Epo

In [10]:
# Model with Dilation convolution
model_dilation = keras.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', dilation_rate=(2,2), input_shape=(28,28,3)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', dilation_rate=(2,2)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', dilation_rate=(2,2)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=512, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=26, activation="softmax")
])

model_dilation.compile(optimizer="Adam", loss="SparseCategoricalCrossentropy", metrics=["sparse_categorical_accuracy"])

In [11]:
# Train the model with dilation convolution
history_dilation = model_dilation.fit(x=X_train, y=y_train, epochs=40, validation_split=0.3)

Epoch 1/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 207ms/step - loss: 4.0530 - sparse_categorical_accuracy: 0.1834 - val_loss: 3.3570 - val_sparse_categorical_accuracy: 0.0533
Epoch 2/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 149ms/step - loss: 1.7077 - sparse_categorical_accuracy: 0.5427 - val_loss: 4.3991 - val_sparse_categorical_accuracy: 0.0427
Epoch 3/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 160ms/step - loss: 0.9164 - sparse_categorical_accuracy: 0.7187 - val_loss: 5.8498 - val_sparse_categorical_accuracy: 0.0427
Epoch 4/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 149ms/step - loss: 0.5984 - sparse_categorical_accuracy: 0.8249 - val_loss: 8.3803 - val_sparse_categorical_accuracy: 0.0427
Epoch 5/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step - loss: 0.3303 - sparse_categorical_accuracy: 0.9131 - val_loss: 10.8200 - val_sparse_categorical_accuracy: 0.0427
Epoch 6/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 150ms/step - loss: 0.2499 - sparse_categorical_accuracy: 0.9237 - val_loss: 12.2955 - val_sparse_categorical_accuracy: 0.0427
E

In [12]:
# Evaluate models
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Basic CNN Test Loss:', test_loss)
print('Basic CNN Test Accuracy:', test_acc)

test_loss, test_acc = model_dilation.evaluate(X_test, y_test)
print('Dilation CNN Test Loss:', test_loss)
print('Dilation CNN Test Accuracy:', test_acc)

test_loss, test_acc = model_transpose.evaluate(X_test, y_test)
print('Transpose CNN Test Loss:', test_loss)
print('Transpose CNN Test Accuracy:', test_acc)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.9196 - sparse_categorical_accuracy: 0.8653
Basic CNN Test Loss: 0.9334548711776733
Basic CNN Test Accuracy: 0.8685897588729858
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 1.4800 - sparse_categorical_accuracy: 0.8004
Dilation CNN Test Loss: 1.3523515462875366
Dilation CNN Test Accuracy: 0.8205128312110901
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.9530 - sparse_categorical_accuracy: 0.8561
Transpose CNN Test Loss: 0.9197992086410522
Transpose CNN Test Accuracy: 0.8621794581413269
